In [1]:
import pandas as pd
import polars as pl
from pathlib import Path
import util

pd.set_option('display.float_format', '{:,.1f}'.format)

## Regional Emissions
Only includes light, medium, and heavy vehicles (bus vehicles are excluded). TNC emissions can be included, depending on model settings reported below.

In [2]:
print(f"Include TNC & Taxi Emissions: {util.input_config['include_tnc_emissions']}")

Include TNC & Taxi Emissions: False


In [3]:
emissions_summary = pd.read_csv(util.output_path / 'emissions/emissions_summary.csv')

network = util.process_network_summary()

In [4]:
df_emissions_summary = emissions_summary.copy()

cols_dict = {'pollutant_name': 'Pollutant', 
             'veh_type': 'Vehicle Type',
             'start_tons': 'Start', 
             'intrazonal_tons': 'Intrazonal', 
             'interzonal_tons': 'Interzonal',
             'total_daily_tons': 'Total Daily (Tons)'}
cols = ['Start', 'Intrazonal','Interzonal', 'Total Daily (Tons)']
df_emissions_summary.rename(columns = cols_dict, inplace=True)


In [5]:
df = df_emissions_summary[df_emissions_summary['Vehicle Type'].isin(['light','medium','heavy'])].copy()
df = df.groupby('Pollutant').sum()
df.rename(columns = cols_dict, inplace=True)
df = df.loc[['CO','NOx','PM25 Total','PM10 Total','CO2 Equivalent','VOCs']]

# FIXME line below is failing at 3.11. I dont see a need for it since there are no decimals in the output.
#df = df.applymap(lambda x: x if x > 100 else str(round(x,1)))
df[cols]

,Start,Intrazonal,Interzonal,Total Daily (Tons)
Pollutant,,,,
CO,6.9,0.2,20.6,27.6
NOx,0.7,0.0,3.7,4.4
PM25 Total,0.1,0.0,0.7,0.8
PM10 Total,0.1,0.1,5.2,5.3
CO2 Equivalent,163.8,23.1,"6,352.9","6,539.7"
VOCs,0.5,0.0,0.4,0.9


## Results by Vehicle Type

### VMT

In [6]:
df_network = network.copy()

df_network['@lveh'] = df_network[['@hov2_inc1','@hov2_inc2', '@hov2_inc3', 
                                  '@hov3_inc1', '@hov3_inc2', '@hov3_inc3',
                                  '@sov_inc1', '@sov_inc2', '@sov_inc3', 
                                  '@tnc_inc1', '@tnc_inc2','@tnc_inc3']].sum(axis=1)

df_network['light'] = df_network['@lveh']*df_network['length']
df_network['medium'] = df_network['@mveh']*df_network['length']
df_network['heavy'] = df_network['@hveh']*df_network['length']

index_labels = ['light','medium','heavy']
df = pd.DataFrame(index=index_labels)
df['VMT'] = df_network[index_labels].sum()

df.index.name = 'Vehicle Type'
df

,VMT
Vehicle Type,
light,"91,273,141.1"
medium,"3,574,973.3"
heavy,"3,550,624.9"


### Emissions

In [7]:
# Calculate emissions and VMT by vehicle type and save results
# Note that Total VMT will not match regional totals because we are not included buses in the emissions summaries

df = df_emissions_summary.copy()
df = df.groupby(['Pollutant','Vehicle Type']).sum()
df.rename(columns = cols_dict, inplace=True)

df.loc[['CO','NOx','PM25 Total','PM10 Total','CO2 Equivalent','VOCs']][cols].copy()


Start  Intrazonal  Interzonal  Total Daily (Tons)
Pollutant      Vehicle Type                                                   
CO             heavy           0.0         0.0         4.0                 4.1
               light           5.7         0.1        15.4                21.2
               medium          1.2         0.0         1.1                 2.3
               transit         0.0         0.0         2.0                 2.0
NOx            heavy           0.0         0.0         3.0                 3.1
               light           0.4         0.0         0.4                 0.8
               medium          0.3         0.0         0.3                 0.6
               transit         0.0         0.0         0.2                 0.2
PM25 Total     heavy           0.0         0.0         0.1                 0.1
               light           0.1         0.0         0.6                 0.6
               medium          0.0         0.0         0.0                 0.1
               transit         0.0         0.0         0.0                 0.0
PM10 Total     heavy           0.0         0.0         0.6                 0.6
               light           0.1         0.1         4.3                 4.4
               medium          0.0         0.0         0.4                 0.4
               transit         0.0         0.0         0.1                 0.1
CO2 Equivalent heavy           1.9         2.3     3,342.6             3,346.8
               light         125.5        19.9     2,209.1             2,354.5
               medium         36.3         0.9       801.2               838.4
               transit         0.8         0.0       466.5               467.3
VOCs           heavy           0.0         0.0         0.1                 0.1
               light           0.3         0.0         0.3                 0.6
               medium          0.2         0.0         0.0                 0.2
               transit         0.0         0.0         0.0                 0.0